# Problem

The Data engineering team often has to deal with dependency of redshift.  This is very important because you cannot drop a view with dropping everything that depends on it. 
Assume you have a table with all the views and table of redshift.

Given that you want to drop a view “VIEW NAME”,  
write a code that generates the order of which   
we need to drop all the views that dependent on the view.  
The “source” view depends on the “dependent” view or table.

# Solution

| source | dependent |
|---|---|
| A | B|
| A | C|
|B | D|


So let me restate the problem as I understand it. Given the table above, We cannot delete B or C until A is deleted. Because A depends on B and C, and we cannot drop a view without first dropping everything that depends on it. By the same logic, we cannot delete D until B and also A (because it is a source that depends on B) is deleted. If we were given 'A' as the view_name argument for our function, we would want it to return ['A']. If we were given 'D' as the view name argument for our function, we would want it to return ['A', 'B', 'D']

This seems like an example of a topological sort, so our function should convert the dataframe into a graph data structure and then perform a depth first search on that graph.

In [1]:
import pandas as pd
from collections import defaultdict, deque

In [2]:
def dfs(graph, start, visited, dependency_order):
    if start not in visited:
        visited.add(start)
        for view in graph[start]:
            dfs(graph, view, visited, dependency_order)
        dependency_order.append(start)

In [3]:
def get_view_dependency(df, view_name):
    """
        df: A dataframe with the table ‘Dependency’
        view_name: the view that we want to drop
    
    OUTPUT:
        List : An ordered list of all the view dependency.  Row n should not depend on anything after that row.
    """
    dependency_graph = defaultdict(list)
    for index, row in df.iterrows():
        dependency_graph[row['dependent']].append(row['source'])
    # Now we want to perform a depth first search of the graph
    visited = set()
    views_to_delete = []
    dfs(dependency_graph, view_name, visited, views_to_delete)
    return views_to_delete

In [4]:
example_dependency = {
    "source": [1,2,2,1,1,3,9,7,6,1,12,11,5],
    "dependent": [2,3,4,3,5,7,10,11,12,6,13,13,6],
}
dependency_df = pd.DataFrame(data=example_dependency)

In [5]:
dependency_graph = defaultdict(list)
for index, row in dependency_df.iterrows():
    dependency_graph[row['dependent']].append(row['source'])
print(dependency_graph)

defaultdict(<class 'list'>, {2: [1], 3: [2, 1], 4: [2], 5: [1], 6: [1, 5], 7: [3], 10: [9], 11: [7], 12: [6], 13: [12, 11]})


In this example, 1 depends on 2,3, and 5. 2 depends on 3 and 4. And so on...
So we cannot drop 2,3, or 5 prior to dropping 1.

In [6]:
print(dependency_df)

    dependent  source
0           2       1
1           3       2
2           4       2
3           3       1
4           5       1
5           7       3
6          10       9
7          11       7
8          12       6
9           6       1
10         13      12
11         13      11
12          6       5


In [7]:
#expect this to return [1]
get_view_dependency(dependency_df, 1)

[1]

In [8]:
#expect this to return [1,2,3]
get_view_dependency(dependency_df, 3)

[1, 2, 3]

In [9]:
#expect this to return [1, 5, 6, 12, 2, 3, 7, 11, 13]
get_view_dependency(dependency_df, 13)

[1, 5, 6, 12, 2, 3, 7, 11, 13]

In [10]:
original_dependency = {
    "source": ["A","A","B"],
    "dependent": ["B", "C", "D"],
}

In [11]:
new_df = pd.DataFrame(data=original_dependency)
print(new_df)

  dependent source
0         B      A
1         C      A
2         D      B


In [12]:
# should return ['A']
get_view_dependency(new_df, "A")

['A']

In [17]:
# should return ['A', 'B', 'D']
get_view_dependency(new_df, "D")

['A', 'B', 'D']

# Extra Credit Problem

How do generate a query that can drop and create the views?

In [14]:
def generate_view_drop_sql(view_names):
    drop_view_template = "DROP VIEW IF EXISTS {};"
    start = "begin;"
    end = "end;"
    
    all_drop_statements = []
    for view_name in view_names:
        all_drop_statements.append(drop_view_template.format(view_name))
        
    full_query = start + " ".join(all_drop_statements) + end
    return full_query

In [15]:
generate_view_drop_sql(["users", "subscriptions"])

'begin;DROP VIEW IF EXISTS users; DROP VIEW IF EXISTS subscriptions;end;'

In [16]:
generate_view_drop_sql(get_view_dependency(new_df, "D"))

'begin;DROP VIEW IF EXISTS A; DROP VIEW IF EXISTS B; DROP VIEW IF EXISTS D;end;'